# Demo for LumiSpy package working with AttoLight SEM CL data

This notebook shows:

- Loading a `HYPCard.bin` file correctly with the lumispy load function
- Plotting cathodoluminescence data in different ways
- Dealing with metadata
- Correcting for spectral defects

Import packages:

In [2]:
%matplotlib qt
#For pop-up window plots, with interactive functionalities. If error, use instead %matplotlib tk 
import lumispy as lum

## Loading HYPCard files

Use the `load_hypcard` function to load the AttoLight SEM specific files. State the relative path to the `HYPCard.bin` file.

*You can also leave the path empty. A pop-up window will appear to select the `HYPCard.bin` file from the browser.*

In [32]:
path = "DemoSample_fromAttoLightSEM/HYPCard.bin"
cl_sem = lum.load_hypcard(path)
cl_sem

<CLSEMSpectrum, title: , dimensions: (64, 64|1024)>

In [31]:
cl_sem.plot()

## Correction of spectral defects

Let's start some post-processing methods.

### Background removal

Check if a background was saved during image acquisiton (in the AttoLight system). If so, it has been stored automatically in the signal object.

Otherwise, manually load a background file, using `np.loadtxt(path_to_file)`.

In [33]:
cl_sem.background != None

True

Run background substraction. No arguments required if the background is stored in the signal object. Otherwise, a background array of shape `[wavelength, bacgkround intensity]` needs to be passed as a `background` argument.

Optional arguments are `background`: to substract an external background file, `inplace`: if `True` it will overwrite the original signal, if `False` if outputs a copy of the signal, with the background substracted (this is by default).

In [34]:
cl_sem_bkg = cl_sem
cl_sem_bkg = cl_sem.background_substraction()

### Correction of acquisiton defects

Correct for the intrinsic shift caused by the misalignment of the grating with the spectrometer aperture centre. For the AttoLight in Cambridge, we have created a function to correct for it

In [35]:
cl_sem_bkg.correct_grating_shift()

The edges of the scan get higher intensities, so they can be cropped. 
If you set the `inplace` parameter to `True` the original CLSEMSpectrum object will be modified, if `False` a cropped copy of it will be created.

In [8]:
cl_sem_bkg = cl_sem_bkg.crop_edges(crop_px=5)

There are also corrections for the cosmic rays (pixels with sharp spikes):

In [36]:
cl_sem_bkg_spikes = cl_sem_bkg.cosmic_rays_substraction(extra_percent=50)

## Plotting data

Plot the corrected data:

### Panchromatic image:

In [62]:
cl_sem_bkg_spikes.T.mean().plot(cmap='viridis')

Plot the average CL spectrum:

In [57]:
cl_sem_bkg_spikes.mean().plot()

### CL signal with the SEM image as navigator axis

Plot with the SEM image instead of the panchromatic navigation image.

*Note: if you cropped the edges before, you'll also need to crop the SEM image*

In [60]:
#Find the folder, path of the SE_Scan file and load the file
import os
se_path = os.path.join(os.path.split(path)[0], "SE_Scan*.png")
SE = lum.hyperspyload(se_path)

In [61]:
#Crop the SEM image (if you have cropped the edges beforehand) and plot
try:
    crop = cl_sem_bkg_spikes.metadata.Signal.cropped_edges
    size_x = cl_sem_bkg_spikes.metadata.Acquisition_instrument.SEM.resolution_x
    size_y = cl_sem_bkg_spikes.metadata.Acquisition_instrument.SEM.resolution_x
    SE = SE.isig[crop:size_x-crop, crop:size_y-crop]
    cl_sem_bkg_spikes.plot(navigator=SE)
#Simply plot with the SEM image
except:
    cl_sem_bkg_spikes.plot(navigator=SE)

## Metadata

All the microscope & spectrometer parameters are saved in the metadata, as well as the data processing steps applied. It can be assessed via:

In [19]:
cl_sem_bkg_spikes.metadata

├── Acquisition_instrument
│   ├── CCD
│   │   ├── amplification = 1
│   │   ├── binning = 1
│   │   ├── channels = 1024
│   │   ├── exposure_time_s = 0.3
│   │   └── readout_rate_khz = 1
│   ├── SEM
│   │   ├── FOV = 26391.201172
│   │   ├── aperture_um = 100.0
│   │   ├── beam_acc_voltage_kv = 5.998
│   │   ├── chamber_pressure_torr = 1.0426e-07
│   │   ├── dwell_time_scan_s = 0.30000001200000004
│   │   ├── gun_lens_amps = 1.2
│   │   ├── obj_lens_amps = 0.325544
│   │   ├── real_magnification = 26391.2011
│   │   ├── resolution_x = 64
│   │   └── resolution_y = 64
│   ├── Spectrometer
│   │   ├── central_wavelength_nm = 700.01
│   │   └── grating = 150.0
│   └── acquisition_system = cambridge_attolight
├── General
│   └── title = 
└── Signal
    ├── background_substracted = True
    ├── binned = False
    ├── cosmic_rays_substracted_extra_percent = 50
    ├── cropped_edges = 5
    ├── grating_corrected = True
    └── signal_type = cl_sem_spectrum